# US Traffic Accidents Visualization

Interactive D3.js visualization showing US traffic accident data by state with detailed road feature breakdowns.

## Instructions
1. Place your CSV file in the `archive/` folder
2. Run all cells in this notebook
3. The visualization will appear at the bottom

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import json
from IPython.display import HTML, display
import http.server
import socketserver
import threading
import time
import matplotlib.pyplot as plt
import seaborn as sns

## Data Processing

In [ ]:
# Find CSV file in archive folder
csv_files = glob.glob('archive/*.csv')
if not csv_files:
    print("Error: No CSV file found in archive/ folder")
    print("Please place your CSV file in the archive/ folder")
else:
    csv_file = csv_files[0]
    print(f"Loading traffic data from: {csv_file}")
    df = pd.read_csv(csv_file)
    
    print(f"Original data shape: {df.shape}")
    print(f"\nAll column names:")
    print(df.columns.tolist())

In [ ]:
# Select columns for analysis
columns_needed = ['Severity', 'State', 'City', 'Start_Time', 'Weather_Condition', 
                 'Civil_Twilight', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 
                 'Junction', 'No_Exit', 'Railway', 'Stop', 'Traffic_Signal']

# Check which columns exist
available_cols = [col for col in columns_needed if col in df.columns]
print(f"Available columns: {available_cols}")

df_sample = df[available_cols].dropna().sample(n=50000, random_state=42)
print(f"Sample data shape: {df_sample.shape}")

In [ ]:
# Process road features data
road_features = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Stop', 'Traffic_Signal']
existing_features = [col for col in road_features if col in df_sample.columns]

# Create state-level data for map
state_totals = df_sample.groupby('State').size().reset_index(name='total_accidents')
print("State totals created")

# Create state-level heatmap data
state_heatmaps = {}
for state in df_sample['State'].unique():
    state_data = df_sample[df_sample['State'] == state]
    heatmap_data = []
    
    for twilight in ['Day', 'Night']:
        twilight_data = state_data[state_data['Civil_Twilight'] == twilight]
        row = {'twilight': twilight}
        
        for feature in existing_features:
            count = len(twilight_data[twilight_data[feature] == True]) if len(twilight_data) > 0 else 0
            row[feature] = count
        
        heatmap_data.append(row)
    
    state_heatmaps[state] = heatmap_data

print("Heatmap data created")

In [ ]:
# Save files for visualization
state_totals.to_csv('state_totals.csv', index=False)

with open('state_heatmaps.json', 'w') as f:
    json.dump(state_heatmaps, f)

print("Data processed successfully!")
print("Files created: state_totals.csv, state_heatmaps.json")

# Show basic stats
print("\nData overview:")
print(f"Civil Twilight distribution:\n{df_sample['Civil_Twilight'].value_counts()}")
print(f"\nTop 10 states by accidents:\n{state_totals.sort_values('total_accidents', ascending=False).head(10)}")

## Static Visualizations

These static plots show key insights from the traffic accident data and will display in GitHub's notebook preview.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('default')
sns.set_palette('viridis')

# Create figure with subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Top 15 States by Accidents
top_states = state_totals.sort_values('total_accidents', ascending=False).head(15)
ax1.bar(top_states['State'], top_states['total_accidents'], color='steelblue')
ax1.set_title('Top 15 States by Traffic Accidents', fontsize=14, fontweight='bold')
ax1.set_xlabel('State')
ax1.set_ylabel('Total Accidents')
ax1.tick_params(axis='x', rotation=45)

# 2. Day vs Night Distribution
twilight_counts = df_sample['Civil_Twilight'].value_counts()
colors = ['#ffd700', '#191970']
ax2.pie(twilight_counts.values, labels=twilight_counts.index, autopct='%1.1f%%', colors=colors)
ax2.set_title('Day vs Night Accident Distribution', fontsize=14, fontweight='bold')

# 3. Severity Distribution
severity_counts = df_sample['Severity'].value_counts().sort_index()
ax3.bar(severity_counts.index, severity_counts.values, color='coral')
ax3.set_title('Accident Severity Distribution', fontsize=14, fontweight='bold')
ax3.set_xlabel('Severity Level')
ax3.set_ylabel('Number of Accidents')

# 4. Road Features Heatmap
feature_summary = []
for feature in existing_features:
    day_count = len(df_sample[(df_sample[feature] == True) & (df_sample['Civil_Twilight'] == 'Day')])
    night_count = len(df_sample[(df_sample[feature] == True) & (df_sample['Civil_Twilight'] == 'Night')])
    feature_summary.append([day_count, night_count])

heatmap_data = np.array(feature_summary).T
sns.heatmap(heatmap_data, xticklabels=[f.replace('_', ' ') for f in existing_features], 
           yticklabels=['Day', 'Night'], annot=True, fmt='d', cmap='Blues', ax=ax4)
ax4.set_title('Road Features by Time of Day', fontsize=14, fontweight='bold')
ax4.set_xlabel('Road Features')
plt.setp(ax4.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig('traffic_analysis_summary.png', dpi=300, bbox_inches='tight')
plt.show()

## Interactive Visualization

The visualization below shows an interactive US map with traffic accident data. Hover over states to see detailed breakdowns.

In [ ]:
# Read the HTML and JS files
with open('index.html', 'r') as f:
    html_content = f.read()

with open('visualization.js', 'r') as f:
    js_content = f.read()

# Embed the visualization directly in the notebook
embedded_html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Traffic Data Visualization</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://unpkg.com/topojson@3"></script>
    <style>
        body {{ 
            font-family: Arial, sans-serif; 
            margin: 20px;
        }}
        .chart {{ 
            border: 1px solid #ccc; 
            margin: 20px 0;
        }}
        .bar {{ 
            fill: steelblue; 
        }}
        .bar:hover {{ 
            fill: orange; 
        }}
        .axis {{ 
            font-size: 12px; 
        }}
        .title {{ 
            font-size: 16px; 
            font-weight: bold; 
            text-anchor: middle;
        }}
        .state {{
            cursor: pointer;
        }}
        .state:hover {{
            stroke: #333;
            stroke-width: 2;
        }}
        #state-chart {{
            background: #f9f9f9;
            border-radius: 5px;
        }}
        .state-bar {{
            fill: steelblue;
            cursor: pointer;
        }}
        .state-bar:hover {{
            fill: orange;
        }}
        .bar-day {{
            fill: #ffd700;
        }}
        .bar-night {{
            fill: #191970;
        }}
        .axis-text {{
            font-size: 11px;
        }}
        .chart-title {{
            font-size: 14px;
            font-weight: bold;
            text-anchor: middle;
        }}
    </style>
</head>
<body>
    <h1>US Traffic Accidents: Interactive State Map</h1>
    
    <div id="us-map"></div>
    <div id="state-popup" style="position: absolute; background: white; border: 2px solid #333; border-radius: 10px; padding: 15px; display: none; box-shadow: 0 4px 12px rgba(0,0,0,0.4); z-index: 1000; width: 400px;"></div>
    <div id="tooltip" style="position: absolute; opacity: 0; background: rgba(0,0,0,0.8); color: white; padding: 10px; border-radius: 5px; pointer-events: none;"></div>
    
    <script>
    {js_content}
    </script>
</body>
</html>
"""

# Display the visualization
display(HTML(embedded_html))